In [70]:
import pandas as pd
chromosome = "1A"
path_blast = '../../../data/TEs/blast/' + chromosome + '.fasta.csv'
path_blast_filtered = '../../../data/TEs/blast/' + chromosome + '.filtered.csv'


In [71]:
#TEs
params = {'min_len':50,'max_len':False,'min_distance':5,'max_q':1.3,'min_q':0.7,'min_pident':80,'min_qcov':50}


In [72]:
#read blast output
df = pd.read_csv(path_blast, sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
print('initial:',len(df.index))
initial = len(df.index)

initial: 3346762


In [73]:
#filter by length
if(params['min_len']):
    df = df[df.qlen > params['min_len']]
print('Min len: ' + str(len(df.index)))
min_length = str(len(df.index))

Min len: 3346743


In [74]:
if(params['max_len']):
    df = df[df.qlen < params['max_len']]
print('Max len: ' + str(len(df.index)))    
max_length = str(len(df.index))

Max len: 3346743


In [75]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params['min_q'])]
print('min treshold:',len(df.index))
min_treshold = str(len(df.index))

min treshold: 789036


In [76]:
df = df[((df.length / df.qlen) <= params['max_q'])]
print('max treshold:',len(df.index))
max_treshold = str(len(df.index))

max treshold: 789036


In [77]:
#filter by pident
df = df[(df.pident >= params['min_pident'])]
print('Min_pident: ' + str(len(df.index)))
min_pident = str(len(df.index))

Min_pident: 733546


In [78]:
#filter by qcov
df = df[(df.qcovs >= params['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))
min_qcov = str(len(df.index))

Min qcov: 733546


In [83]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df = df.sort_values(by=['sseqid','sstart', 'send'])
df = df.reset_index(drop=True)
# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]

In [90]:
df.head(10)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
0,MITE_T_127050|chr3D|246631828|246632385|CT|10|...,1A,1,481,77566,78048,62,28,81.855,2.040000e-107,496,557,594102056,90
1,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,43,517,77566,78050,31,14,90.760,0.000000e+00,487,569,594102056,89
2,MITE_T_122702|chr2B|476704370|476705072|AT|10|...,1A,95,698,77566,78177,78,22,83.845,1.120000e-160,619,702,594102056,100
3,MITE_T_127050|chr3D|246631828|246632385|CT|10|...,1A,1,479,86285,86765,65,26,81.542,1.230000e-104,493,557,594102056,90
4,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,44,515,86285,86766,32,14,90.496,5.260000e-178,484,569,594102056,89
5,MITE_T_10416|chr4A|373381651|373381997|TGTT|36...,1A,7,341,213862,214198,20,8,91.765,7.150000e-130,340,346,594102056,98
6,MITE_T_8595|chr2D|162421741|162422102|TTT|124|...,1A,38,361,213865,214188,53,10,80.851,8.210000e-65,329,361,594102056,100
7,MITE_T_9379|chr5A|253680838|253681189|GC|364|F534,1A,12,340,213865,214195,24,4,91.566,1.570000e-126,332,351,594102056,99
8,MITE_T_10416|chr4A|373381651|373381997|TGTT|36...,1A,10,338,213865,214196,46,11,83.036,3.570000e-78,336,346,594102056,98
9,MITE_T_11550|chr4B|491446578|491446917|ATTC|34...,1A,3,335,213865,214197,47,8,83.680,3.470000e-83,337,339,594102056,100


In [92]:
# filter overlapped 
rows = []
discard = []
total = len(df.index)
count = 0
curr = 0
for index, row in df.iterrows():
    count += 1
    curr_new = int(count * 100 * 1.0 / (total * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 1 == 0:
            print(curr_new)
    if index in discard:
        continue
    for k2, v2 in df.loc[index:,].iterrows():
        if abs(v2.sstart - row.sstart) > params['min_distance']:
            break
        if abs(v2.sstart - row.sstart) <= params['min_distance'] and abs(v2.send - row.send) <= params['min_distance']:
            discard.append(k2)
    rows.append(row)


1
2
3


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(rows)
print('Non overlapped: ' + str(len(df.index)))
non_overlapped = str(len(df.index))


In [ ]:
filename = path_blast + params['file'] + '.myfiltered'
df.to_csv(path_blast_filtered, index=None, sep='\t')
filename

In [ ]:
print('Initial: ' + str(initial))
print('Min len: ' + str(min_length))
print('Max len: ' + str(max_length))
print('Min treshold: ' + str(min_treshold))
print('Max treshold: ' + str(max_treshold))
print('Min pident: ' + str(min_pident))
print('Min qcov: ' + str(min_qcov))
print('Non overlapped: ' + str(non_overlapped))
print('Saved: ' + path_blast_filtered)